# Fine Tuning ( Resnet head no aug / mobilenet mid aug)

In this notebook we demonstrates how to run the entire experimental pipeline (Hyperparameter Optimization, Final Training, and Evaluation) for a model variation that will be selected in a later cell. We achive this by calling the core functions defined in `hpo.py`, `final_training.py`, and `evaluation.py`.

### Selecting a model
Due to time contraints we have the models in list that will be ran as batches. This will be changed later. For now please only select one of the following cells.

In [1]:
SETUP_IDS_TO_RUN = ['resnet_head_noaug', 'mobilenet_mid_aug']


In [4]:
import torch
import os
import time # For potential timing if desired, though scripts handle internal timing
import sys
sys.path.append('../scripts') 

# Import shared utilities and configurations 
import utils
import config # Contains HPO_CONFIG_LIST, UNFREEZE_MAPS, output dirs, defaults

# Import the core processing functions from our scripts
from hpo import perform_hpo_for_setup
from final_train import perform_final_training_for_setup
from evaluate import perform_evaluation_for_setup

# --- Define the Single Setup to Run ---
print(f"### Will run pipeline for Setups ID: {SETUP_IDS_TO_RUN} ###")

# --- Global Parameters for this Run (can override config defaults if needed) ---
# These would typically come from argparse in the scripts, here we set them directly
# Or, we can rely on defaults set within config.py and the script functions
HPO_EPOCHS = config.DEFAULT_HPO_EPOCHS
FINAL_TRAIN_EPOCHS = config.DEFAULT_FINAL_TRAIN_EPOCHS
PATIENCE = config.DEFAULT_PATIENCE
BATCH_SIZE = config.DEFAULT_BATCH_SIZE # Using one batch size for simplicity here, scripts might allow separate

# --- Device Setup ---
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

# --- Ensure Output Directories Exist ---
config.create_output_dirs()

### Will run pipeline for Setups ID: ['resnet_head_noaug', 'mobilenet_mid_aug'] ###
Using device: cuda
Output directories ensured.


## Stage 1: Hyperparameter Optimization

In [9]:
print(f"\n{'='*5} STAGE 1: Hyperparameter Optimization {'='*5}")

for setup_id in SETUP_IDS_TO_RUN:
    print(f"\nRunning HPO for setup: {setup_id}")
    hpo_start_time = time.time()

    # Call the HPO function from run_hpo.py
    best_hpo_params_found = perform_hpo_for_setup(
        setup_id=setup_id,
        hpo_grid=config.HPO_CONFIG_LIST,
        num_epochs_hpo=HPO_EPOCHS,
        batch_size=BATCH_SIZE,
        device=device
        # The function saves its outputs to files within config.HPO_DIR
    )

    hpo_end_time = time.time()
    print(f"\n{'*'*10} HPO Stage for {setup_id} finished in {(hpo_end_time - hpo_start_time)/60:.2f} minutes {'*'*10}")

    if best_hpo_params_found:
        print(f"Best HPO parameters found for {setup_id}: {best_hpo_params_found}")
        # These are also saved to results/hpo/best_params_{setup_id}.json
    else:
        print(f"HPO did not complete successfully or find best parameters for {setup_id}. Cannot proceed.")
        # Consider raising an error or stopping the notebook here if HPO is critical for next steps



===== STAGE 1: Hyperparameter Optimization =====

Running HPO for setup: resnet_head_noaug
--- Starting HPO for Setup ID: resnet_head_noaug ---
  Trial 1/12 with params: {'lr_backbone': 0.0001, 'lr_head': 0.005, 'weight_decay': 0.01}


100%|██████████| 792M/792M [02:01<00:00, 6.54MB/s]  


Extracting data/oxford-iiit-pet/images.tar.gz to data/oxford-iiit-pet


100%|██████████| 19.2M/19.2M [00:04<00:00, 4.59MB/s]


Extracting data/oxford-iiit-pet/annotations.tar.gz to data/oxford-iiit-pet
    Adjusting lr_backbone to 0 for 'head' unfreeze strategy.


...

    Epoch 11/15 - Train Loss: 0.1874, Acc: 95.99% | Val Loss: 0.5528, Acc: 82.88%


    Epoch 12/15 - Train Loss: 0.1600, Acc: 96.84% | Val Loss: 0.5558, Acc: 83.02%


    Epoch 13/15 - Train Loss: 0.1485, Acc: 97.38% | Val Loss: 0.5466, Acc: 83.42%


    Epoch 14/15 - Train Loss: 0.1417, Acc: 96.94% | Val Loss: 0.5425, Acc: 83.83%


    Epoch 15/15 - Train Loss: 0.1277, Acc: 97.38% | Val Loss: 0.5425, Acc: 83.42%
    New best HPO params for mobilenet_mid_aug: {'lr_backbone': 5e-05, 'lr_head': 0.001, 'weight_decay': 0} with Val Acc: 83.42%
  Full HPO history for mobilenet_mid_aug saved to: ../results/hpo/hpo_history_mobilenet_mid_aug.json
  Best hyperparameters for mobilenet_mid_aug saved to: ../results/hpo/best_params_mobilenet_mid_aug.json
--- Finished HPO for Setup ID: mobilenet_mid_aug ---

********** HPO Stage for mobilenet_mid_aug finished in 22.84 minutes **********
Best HPO parameters found for mobilenet_mid_aug: {'lr_backbone': 5e-05, 'lr_head': 0.001, 'weight_decay': 0}


## Stage 2: Final Model Training

In [10]:
for setup_id in SETUP_IDS_TO_RUN:
    if best_hpo_params_found:  # Proceed only if HPO was successful
        print(f"\n{'='*20} STAGE 2: Final Model Training for {setup_id} {'='*20}")
        final_train_start_time = time.time()

        # Call the final training function from run_final_training.py
        # It will load the best_params file itself based on setup_id.
        saved_model_filepath = perform_final_training_for_setup(
            setup_id=setup_id,
            max_epochs=FINAL_TRAIN_EPOCHS,
            patience=PATIENCE,
            batch_size=BATCH_SIZE,
            device=device
            # The function saves its outputs to files within config.FINAL_TRAINING_DIR
        )

        final_train_end_time = time.time()
        print(f"\n{'*'*10} Final Training Stage for {setup_id} finished in {(final_train_end_time - final_train_start_time)/60:.2f} minutes {'*'*10}")

        if saved_model_filepath:
            print(f"Best model saved to: {saved_model_filepath}")
            # This path is also saved internally by the function to results/final_training/best_model_{setup_id}.pth
        else:
            print(f"Final training did not complete successfully or model was not saved for {setup_id}. Cannot proceed to evaluation.")
    else:
        print(f"\nSkipping Stage 2: Final Model Training for {setup_id} due to HPO failure or no best parameters found.")
        saved_model_filepath = None  # Ensure it's None if HPO failed


==================== STAGE 2: Final Model Training for resnet_head_noaug ====================
--- Starting Final Training for Setup ID: resnet_head_noaug ---
  Loaded best HPO params: {'lr_backbone': 5e-05, 'lr_head': 0.001, 'weight_decay': 0.001}
  Epoch 1/50


    Train Loss: 2.2411 Acc: 50.35% | Time: 6.14s
    Training loss improved (inf -> 2.2411). Saving model...
    Best model (lowest training loss) weights saved to ../results/final_training/best_model_resnet_head_noaug.pth
  Epoch 2/50


    Train Loss: 0.9139 Acc: 84.40% | Time: 6.00s
    Training loss improved (2.2411 -> 0.9139). Saving model...
    Best model (lowest training loss) weights saved to ../results/final_training/best_model_resnet_head_noaug.pth
  Epoch 3/50


    Train Loss: 0.6068 Acc: 88.26% | Time: 6.15s
    Training loss improved (0.9139 -> 0.6068). Saving model...
    Best model (lowest training loss) weights saved to ../results/final_training/best_model_resnet_head_noaug.pth
  Epoch 4/50


    Train Loss: 0.4825 Acc: 90.14% | Time: 6.03s
    Training loss improved (0.6068 -> 0.4825). Saving model...
    Best model (lowest training loss) weights saved to ../results/final_training/best_model_resnet_head_noaug.pth
  Epoch 5/50


...

    Train Loss: 0.0276 Acc: 99.48% | Time: 7.91s
    Training loss improved (0.0281 -> 0.0276). Saving model...
    Best model (lowest training loss) weights saved to ../results/final_training/best_model_mobilenet_mid_aug.pth
  Epoch 47/50


    Train Loss: 0.0234 Acc: 99.67% | Time: 7.78s
    Training loss improved (0.0276 -> 0.0234). Saving model...
    Best model (lowest training loss) weights saved to ../results/final_training/best_model_mobilenet_mid_aug.pth
  Epoch 48/50


    Train Loss: 0.0258 Acc: 99.40% | Time: 7.64s
    No improvement in training loss for 1 epoch(s).
  Epoch 49/50


    Train Loss: 0.0213 Acc: 99.59% | Time: 7.82s
    Training loss improved (0.0234 -> 0.0213). Saving model...
    Best model (lowest training loss) weights saved to ../results/final_training/best_model_mobilenet_mid_aug.pth
  Epoch 50/50


    Train Loss: 0.0229 Acc: 99.59% | Time: 7.90s
    No improvement in training loss for 1 epoch(s).
  Training Finished. Total time: 6.52 minutes
  Lowest Training Loss Achieved: 0.0213
  Final training history for mobilenet_mid_aug saved to: ../results/final_training/final_training_history_mobilenet_mid_aug.json
--- Finished Final Training for Setup ID: mobilenet_mid_aug ---

********** Final Training Stage for mobilenet_mid_aug finished in 6.53 minutes **********
Best model saved to: ../results/final_training/best_model_mobilenet_mid_aug.pth


## Stage 3: Evaluation

In [11]:
for setup_id in SETUP_IDS_TO_RUN:
    if saved_model_filepath and os.path.exists(saved_model_filepath):  # Proceed only if final training produced a model
        print(f"\n{'='*20} STAGE 3: Evaluation for {setup_id} {'='*20}")
        eval_start_time = time.time()

        # Call the evaluation function from run_evaluation.py
        # It will load the model file itself based on setup_id and config.FINAL_TRAINING_DIR
        perform_evaluation_for_setup(
            setup_id=setup_id,
            device=device
            # The function saves its outputs to files within config.EVALUATION_DIR
        )

        eval_end_time = time.time()
        print(f"\n{'*'*10} Evaluation Stage for {setup_id} finished in {(eval_end_time - eval_start_time)/60:.2f} minutes {'*'*10}")
        print(f"\nEvaluation results (predictions and metrics CSVs) are saved in: {config.EVALUATION_DIR}")
    else:
        print(f"\nSkipping Stage 3: Evaluation for {setup_id} due to final training failure or model not found.")

/home/julianm56/Dev/Math392/MATH392_Final_Project/notebooks/../scripts/evaluate.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_p


==================== STAGE 3: Evaluation for resnet_head_noaug ====================
--- Starting Evaluation for Setup ID: resnet_head_noaug ---
  Loaded model from ../results/final_training/best_model_resnet_head_noaug.pth


/home/julianm56/Dev/Math392/MATH392_Final_Project/notebooks/../scripts/evaluate.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_p

  Predictions saved to: ../results/evaluation/predictions_resnet_head_noaug.csv
  Evaluation metrics saved to: ../results/evaluation/eval_metrics_resnet_head_noaug.csv
  Overall Test Accuracy for resnet_head_noaug: 86.86%
--- Finished Evaluation for Setup ID: resnet_head_noaug ---

********** Evaluation Stage for resnet_head_noaug finished in 0.30 minutes **********

Evaluation results (predictions and metrics CSVs) are saved in: ../results/evaluation

==================== STAGE 3: Evaluation for mobilenet_mid_aug ====================
--- Starting Evaluation for Setup ID: mobilenet_mid_aug ---
  Loaded model from ../results/final_training/best_model_mobilenet_mid_aug.pth


  Predictions saved to: ../results/evaluation/predictions_mobilenet_mid_aug.csv
  Evaluation metrics saved to: ../results/evaluation/eval_metrics_mobilenet_mid_aug.csv
  Overall Test Accuracy for mobilenet_mid_aug: 80.54%
--- Finished Evaluation for Setup ID: mobilenet_mid_aug ---

********** Evaluation Stage for mobilenet_mid_aug finished in 0.34 minutes **********

Evaluation results (predictions and metrics CSVs) are saved in: ../results/evaluation
